In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

metadata = pd.read_csv("../stock_data_cleaned/stock_vector_and_encoding.csv")

In [3]:
# 定义时间序列拆分器
tscv = TimeSeriesSplit(n_splits=5)
# 初始化一个 100x5 的数组，初始值为 0
split_array = np.zeros((100, 5))

# 进行时间序列拆分
for fold, (train_index, test_index) in enumerate(tscv.split(np.arange(100))):
    split_array[train_index, fold] = 1
    split_array[test_index, fold] = 2

In [5]:
original_path = "stock_data_model"
split_path = "stock_data_model_split"
# 定义时间序列拆分器
tscv = TimeSeriesSplit(n_splits=5)

for file in metadata["Symbol"]:
    data = pd.read_csv("../" + original_path + "/" + file + ".csv")
    split_array = np.zeros((len(data), 5))
    for fold, (train_index, test_index) in enumerate(tscv.split(np.arange(len(data)))):
        split_array[train_index, fold] = 1
        split_array[test_index, fold] = 2
    split_df = pd.DataFrame(
        split_array, columns=["split_0", "split_1", "split_2", "split_3", "split_4"]
    )
    data = pd.concat([data, split_df], axis=1)
    data.to_csv("../" + split_path + "/" + file + ".csv")